# Segmenting and clustering Toronto Neighborhood

Libraries first 

In [27]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup as bs
import requests
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.36 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.52 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.89 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.57 MB/s


Use Beautiful soap to extract data from wikipedia

In [42]:
raw_wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
extract = bs(raw_wiki)
print(extract.prettify())    #Not printing it to keep the notebook short. It works.

We need only data that constitute table. So lets target the table content. 

In [43]:
table = extract.find('table',{'class':'wikitable sortable'})
table
## Not displaying extracts to keep the notebook length short. It works. 

Now we have html markers'tr' and 'td' to identify the fields. 

1.Use Findall to find the text

2.Append to append the text based on conditions

3.Populate list by iterating through the loop 

!!Remember to remove "Not Assigned"


In [5]:
postcode = []
borough = []
neighborhood = []

for row in table.find_all('tr')[1:]:  # Find tr . Leave first row for heading
    borough_list = row.find_all('td')[1] # Get boroughs.  second column
    if (borough_list.text.strip() != 'Not assigned'):  # avoid 'Not assigned'
        borough.append(borough_list.text.strip())    # Append to list borough[]
        postcode_list = row.find_all('td')[0]  #  Find td.  first column
        postcode.append(postcode_list.text.strip()) # Append to list postcode[]
        neighborhood_list = row.find_all('td')[2] # Remaining td . third column
        if (neighborhood_list.text.strip() == 'Not assigned'): 
            neighborhood.append(borough_list.text.strip()) 
        else:
            neighborhood.append(neighborhood_list.text.strip())  #Append to list neighborhood[]


In [37]:
df = pd.DataFrame()
## populate data frame with 3 colums we made.
df['PostalCode'] = postcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood
## df contains repeated values. We use groupby and join

df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [51]:
df.shape


(103, 3)

To get longitudes and latitudes for items in our dataframe

In [13]:
## CSV file provided has list of postcode latitude longitudes. Easier than geocoder method. 
coordinate_df = pd.read_csv('http://cocl.us/Geospatial_data')
coordinate_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df.columns  ## check the column indices

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

Merge the co ordinate data from dataframe coordinate_df to our data frame df

In [38]:
## take each postcode in df and iterate through coordinate_df to find match
## we can also do inner join method. but I prefer loops - easy to understand
for column in df.columns[:1]:
    for column1 in coordinate_df.columns[:1]:
        if df[column][1] == coordinate_df[column1][1]:
            df['Latitude'] = coordinate_df["Latitude"]
            df['Longitude'] = coordinate_df["Longitude"]
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Now the task is //Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto //

In [39]:
df_toronto = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
## df_toronto.shape
df_toronto.head(38)  ## there are 38 Boroughs with Toronto and unique postcode

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


Create Folium map

In [40]:
### We can find lat, long of Toronto from google map.
### Better to find the average of lat long from our dataset
lat_toronto = df_toronto['Latitude'].mean()
long_toronto = df_toronto['Longitude'].mean()
print('Toronto lat {}, Toronto long {}'.format(lat_toronto, long_toronto))

Toronto lat 43.66726218421052, Toronto long -79.38988323421053


In [41]:
# Create map of Toronto centered around the average lat, long
map_toronto = folium.Map(location=[lat_toronto , long_toronto], zoom_start=12)


# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

